In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from amberdata_derivatives import AmberdataDerivatives
from matplotlib.ticker import MaxNLocator
import matplotlib.dates as mdates
import mplfinance as mpf
import numpy as np
import seaborn as sns
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
# insert API key below
amberdata_client = AmberdataDerivatives(api_key="")

amberdata_client.get

In [ ]:
btc = pd.DataFrame(amberdata_client.get_volatility_term_structures_constant(currency='BTC', exchange='deribit')['payload']['data'])
eth = pd.DataFrame(amberdata_client.get_volatility_term_structures_constant(currency='ETH', exchange='deribit')['payload']['data'])

# plot the atm BTC + ETH term structure
plt.figure(figsize=(15, 7))
plt.plot(eth.daysToExpiration, eth.atm, label='ETH')
plt.plot(btc.daysToExpiration, btc.atm, label='BTC')
plt.title("ATM Term Structure Comparison BTC vs. ETH as of " + str(str(pd.to_datetime(btc.timestamp[0]))))
plt.ylabel("Implied Volatility (%)")
plt.xlabel("DTE")
plt.legend()

# plot BTC atm + atm forward vol
plt.figure(figsize=(15, 7))
plt.plot(btc.daysToExpiration, btc.atm, label='Mark')
plt.plot(btc.daysToExpiration, btc.fwdAtm, label='Forward', linestyle='--')
plt.title("ATM Term Structure Comparison BTC Mark vs Forward Curve " + str(str(pd.to_datetime(btc.timestamp[0]))))
plt.ylabel("Implied Volatility (%)")
plt.xlabel("DTE")
plt.legend();

In [ ]:
# plot atm BTC volatility
data = amberdata_client.get_volatility_delta_surfaces_constant(currency='BTC', exchange='deribit', startDate='2024-01-01', endDate='2024-06-01', timeInterval='day')
data = data['payload']['data']
data = pd.DataFrame(data)

date = pd.to_datetime(data[data.daysToExpiration==7].timestamp)

plt.figure(figsize=(15, 7))
plt.plot(date, data[data.daysToExpiration==7].atm, label='7 DTE')
plt.plot(date, data[data.daysToExpiration==30].atm, label='30 DTE')
plt.plot(date, data[data.daysToExpiration==60].atm, label='60 DTE')
plt.plot(date, data[data.daysToExpiration==180].atm, label='180 DTE')
plt.legend()
plt.title("BTC ATM Volatility: 2024-01-01 to 2024-06-01")
plt.ylabel("ATM IV (%)");

In [ ]:
# plot 25 delta BTC skew
data = amberdata_client.get_volatility_delta_surfaces_constant(currency='BTC', exchange='deribit', startDate='2019-04-01', endDate='2022-06-01', timeInterval='day')
data = data['payload']['data']
data = pd.DataFrame(data)

s7 = data[data.daysToExpiration==7]
s30 = data[data.daysToExpiration==30]
s90 = data[data.daysToExpiration==90]

plt.figure(figsize=(15, 7))
plt.plot(s7.deltaCall10 / s7.deltaCall25, label='7 DTE ∆10/∆25')
plt.plot(s30.deltaCall10 / s30.deltaCall25, label='30 DTE ∆10/∆25')
plt.plot(s90.deltaCall10 / s90.deltaCall25, label='90 DTE ∆10/∆25')
plt.axhline(1, linestyle='--', c='r', alpha=0.50)
plt.legend()
plt.title("BTC ∆25/∆35 Ratio [Call - Put]: 2022-01-01 to 2024-06-01")
plt.ylabel("Skew [Call - Put] (%)")

In [ ]:
# plot BTC Deribit vol curves 
data = pd.DataFrame(amberdata_client.get_volatility_level_1_quotes(exchange='deribit', currency='btc')['payload']['data'])
mats = sorted(set(data.expirationTimestamp))

plt.figure(figsize=(15, 7))
for mat in mats:
    sub = data[data.expirationTimestamp == mat]
    sub = sub.sort_values("strike").reset_index(drop=True)
    
    plt.plot(sub.strike, sub.markIv, label=mat[:10])
    plt.legend()
    plt.title("BTC Volatility Curves as of " + str(data.timestamp[0]))
    plt.ylabel("Implied Volatility (%)")
    plt.xlabel("Strike ($)")

In [ ]:
# plot OI for BTC option contracts
data = pd.DataFrame(amberdata_client.get_volatility_level_1_quotes(exchange='deribit', currency='btc')['payload']['data'])
oi = data.groupby("putCall").sum()

plt.figure(figsize=(15, 7))
plt.barh(['Calls'], oi['openInterest']['C']/1_000, color='orange')
plt.barh(['Puts'], oi['openInterest']['P']/1_000, color='blue')
plt.title("BTC Open Interest By Option Type as of " + str(data.timestamp[0]))
plt.xlabel("Option Contracts (Thousands)")

In [ ]:
# plot distribution of BTC option OI across all strikes
data = pd.DataFrame(amberdata_client.get_volatility_level_1_quotes(exchange='deribit', currency='btc')['payload']['data'])

calls = data[data.putCall=='C']
puts = data[data.putCall=='P']

call_strike_oi = calls.groupby("strike").sum()[['openInterest']].reset_index()
put_strike_oi  =  puts.groupby("strike").sum()[['openInterest']].reset_index()

plt.figure(figsize=(15, 7))
sns.barplot(x=call_strike_oi.strike, y=call_strike_oi.openInterest/1000, color='blue', label='Call')
sns.barplot(x=put_strike_oi.strike, y=put_strike_oi.openInterest/1000, color='orange', 
            bottom=call_strike_oi.openInterest/1000, label='Put')

plt.tight_layout()
plt.gca().xaxis.set_major_locator(MaxNLocator(20))
plt.legend()
plt.title("BTC Open Interest By Strike as of " + str(data.timestamp[0]))
plt.ylabel("Open Interest (Thousands of Contracts)")
plt.xlabel("Strike");

In [ ]:
# plot distribution of BTC option OI across all maturities
data = pd.DataFrame(amberdata_client.get_volatility_level_1_quotes(exchange='deribit', currency='btc')['payload']['data'])

calls = data[data.putCall=='C']
puts = data[data.putCall=='P']

calls['expirationTimestamp'] = pd.to_datetime(calls.expirationTimestamp).dt.date
puts['expirationTimestamp']  =  pd.to_datetime(puts.expirationTimestamp).dt.date

c = calls.groupby("expirationTimestamp").sum().reset_index()
p = puts.groupby("expirationTimestamp").sum().reset_index()

c['openInterest'] /= 1000
p['openInterest'] /= 1000

c['op_type'] = 'Call'
p['op_type'] = 'Put'
t = pd.concat([c, p]).reset_index(drop=True)

# Create the bar plot
plt.figure(figsize=(15, 7))
sns.barplot(data=t, x='expirationTimestamp', y='openInterest', hue='op_type')
plt.xticks(rotation=45, ha='right');
plt.title("BTC Open Interest By Expiration as of " + str(data.timestamp[0]))
plt.ylabel("Open Interest (Thousands of Contracts)")
plt.xlabel("Expiry")
plt.legend(title='Option Type')

In [ ]:
# plot 24 hour change in OI 
data = pd.DataFrame(amberdata_client.get_volatility_level_1_quotes(exchange='deribit', currency='btc', startDate='2024-06-05', endDate='2024-06-08', timeInterval='day')['payload']['data'])                                                     
data['timestamp'] = pd.to_datetime(data.timestamp)

yday = data[data.timestamp == '2024-06-06 00:00:00+00:00']
tday = data[data.timestamp == '2024-06-07 00:00:00+00:00']

tday_c = tday[tday.putCall=='C'].groupby("strike")[['openInterest']].sum()
tday_p = tday[tday.putCall=='P'].groupby("strike")[['openInterest']].sum()

yday_c = yday[yday.putCall=='C'].groupby("strike")[['openInterest']].sum()
yday_p = yday[yday.putCall=='P'].groupby("strike")[['openInterest']].sum()

call = pd.concat([tday_c, yday_c], axis=1)
call.columns = ['today', 'yesterday']
call['diffs'] = call.today - call.yesterday

put = pd.concat([tday_p, yday_p], axis=1)
put.columns = ['today', 'yesterday']
put['diffs'] = put.today - put.yesterday

call = call.reset_index()
put  = put.reset_index()

plt.figure(figsize=(15, 7))
sns.barplot(x=call.diffs, y=call.strike,  orient = 'h', color='blue', label='Call')
sns.barplot(x=put.diffs, y=put.strike,  orient = 'h', color='orange', label='Put')
plt.gca().yaxis.set_major_locator(MaxNLocator(10))

plt.title("24hr Change in BTC OI By Strike between " + str(yday.timestamp.iloc[0]) + " to " + str(tday.timestamp.iloc[0]))
plt.ylabel("Strike")
plt.xlabel("Change in OI")
plt.legend(title='Option Type')

In [ ]:
# plot 24 hour change in OI based on maturity
data = pd.DataFrame(amberdata_client.get_volatility_level_1_quotes(exchange='deribit', currency='btc', startDate='2024-06-05', endDate='2024-06-08', timeInterval='day')['payload']['data'])                                                     
data['timestamp'] = pd.to_datetime(data.timestamp)

yday = data[data.timestamp == '2024-06-06 00:00:00+00:00']
tday = data[data.timestamp == '2024-06-07 00:00:00+00:00']

tday_c = tday[tday.putCall=='C'].groupby("expirationTimestamp")[['openInterest']].sum()
tday_p = tday[tday.putCall=='P'].groupby("expirationTimestamp")[['openInterest']].sum()

yday_c = yday[yday.putCall=='C'].groupby("expirationTimestamp")[['openInterest']].sum()
yday_p = yday[yday.putCall=='P'].groupby("expirationTimestamp")[['openInterest']].sum()

calls = pd.concat([tday_c, yday_c], axis=1)
calls.columns = ['today', 'yesterday']
calls['diffs'] = calls.today - calls.yesterday

puts = pd.concat([tday_p, yday_p], axis=1)
puts.columns = ['today', 'yesterday']
puts['diffs'] = puts.today - puts.yesterday

calls['Option Type'] = 'Call'
puts['Option Type'] = 'Put'

t = pd.concat([calls, puts], axis=0).reset_index()
t['expirationTimestamp'] = pd.to_datetime(t.expirationTimestamp).dt.date

plt.figure(figsize=(15, 7))
sns.barplot(data=t, x='diffs', y='expirationTimestamp', hue='Option Type', orient='h')

plt.xticks(rotation=45, ha='right');
plt.title("24hr Change in BTC OI By Maturity between " + str(yday.timestamp.iloc[0]) + " to " + str(tday.timestamp.iloc[0]))
plt.ylabel("Expiration")
plt.xlabel("Change in OI")
plt.legend(title='Option Type')

In [ ]:
# use this endpoint to get back all of the available instruments on deribit
instruments = pd.DataFrame(amberdata_client.get_instruments_information(exchange='deribit')['payload']['data'])
instruments.head(10)

In [ ]:
# Plot BTC <> ETH correlation of log daily returns 
data = amberdata_client.get_realized_volatility_correlation_beta(exchange='gdax', pair='btc_usd', pair2='eth_usd')
data = pd.DataFrame(data['payload']['data'])
data.index = pd.to_datetime(data.timestamp, unit='ms')
data = data[::-1]

plt.figure(figsize=(15, 7))
plt.plot(data.correlation180)
plt.title("BTC <> ETH 180 Day RV Rolling Correlation: " + str(data.index[0]) + " to " + str(data.iloc[-1].name))
plt.ylabel("Rolling Correlation")

In [ ]:
# Plot BTC Monthly vs Daily Realized Volatility 
data = amberdata_client.get_realized_volatility_monthly_vs_daily_ratio(exchange='gdax', pair='btc_usd')
data = pd.DataFrame(data['payload']['data'])
data.index = pd.to_datetime(data.timestamp, unit='ms')

# only look at a subset of data
data = data[:1000][::-1]
plt.figure(figsize=(15, 7))
plt.title("BTC Monthly vs Daily RV: " + str(data.index[0]) + " to " + str(data.iloc[-1].name))
plt.plot(data.monthlyHistoricalVolatility, label='Monthly RV')
plt.plot(data.dailyHistoricalVolatility30Days, label='Daily RV')
plt.ylabel("Realized Volatility (%)")
plt.legend();

In [ ]:
# plot BTC vol seasonality for day of week
data = amberdata_client.get_realized_volatility_seasonality_day_of_week(exchange='gdax', pair='btc_usd', startDate='2024-01-01', endDate='2024-05-01')
data = pd.DataFrame(data['payload']['data'])
data = data[::-1]

plt.figure(figsize=(15, 7))
plt.bar(data.weekday, data.historicalVolatility1day)
plt.ylabel("Average Realized Volatility (%)")
plt.title("BTC Seasonality: Volatility Day of Week: 2024-01-01 to 2024-05-01");

In [ ]:
# plot BTC vol seasonality for month of year
data = amberdata_client.get_realized_volatility_seasonality_month_of_year(exchange='gdax', pair='btc_usd', startDate='2024-01-01', endDate='2024-05-01')
data = pd.DataFrame(data['payload']['data'])

plt.figure(figsize=(15, 7))
plt.bar(data.month, data.historicalVolatility1day)
plt.ylabel("Average Realized Volatility (%)")
plt.title("BTC Seasonality: Volatility Month of Year: 2024-01-01 to 2024-05-01");

In [ ]:
# plot realized volatility cones for BTC 
data = amberdata_client.get_realized_volatility_cones(exchange='gdax', pair='btc_usd', startDate='2024-01-01', endDate='2024-05-01')
data = pd.DataFrame(data['payload']['data'])
data = data.iloc[:, 2:].T

data['dte'] = [
    int(''.join([char for char in column.split("_")[1] if char.isdigit()]))
    for column in data.index
]
data['vol_level'] = [data.index[i].split("_")[0] for i in range(len(data))]
data.columns = ['rv', 'dte', 'vol_level']

cone_max = data[data.vol_level=='max'].sort_values('dte', ascending=True)
cone_min = data[data.vol_level=='min'].sort_values('dte', ascending=True)
cone_25p = data[data.vol_level=='p25'].sort_values('dte', ascending=True)
cone_50p = data[data.vol_level=='p50'].sort_values('dte', ascending=True)
cone_75p = data[data.vol_level=='p75'].sort_values('dte', ascending=True)
cone_current = data[data.vol_level=='current'].sort_values('dte', ascending=True)

plt.figure(figsize=(15, 7))
plt.plot(cone_max.dte, cone_max.rv, label='Maximum')
plt.plot(cone_25p.dte, cone_25p.rv, label='25th Percentile')
plt.plot(cone_50p.dte, cone_50p.rv, label='50th Percentile')
plt.plot(cone_75p.dte, cone_75p.rv, label='75th Percentile')
plt.plot(cone_current.dte, cone_current.rv, label='Current')
plt.plot(cone_min.dte, cone_min.rv, label='Minimum')

plt.legend()
plt.title("BTC RV Volatility Cone: 2024-01-01 to 2024-05-01")
plt.ylabel("Realized Volatility (%)")
plt.xlabel("DTE")

In [ ]:
# plot IV vs RV + VRP 
data = amberdata_client.get_realized_volatility_implied_vs_realized(exchange='deribit', currency='btc', startDate='2024-01-01', endDate='2024-01-07')
data = pd.DataFrame(data['payload']['data'])
data.index = pd.to_datetime(data.timestamp, unit='ms')
data['vrp'] = data.atm7 - data.realizedVolatility7

plt.figure(figsize=(15, 7))
plt.title("BTC ATM IV vs RV [7 DTE]: 2024-01-01 to 2024-01-05")
plt.bar(data.index, data.vrp, label='VRP [ATM - RV]', edgecolor='black', width=0.03)
plt.plot(data.index, data.atm7, label='ATM 7 DTE')
plt.plot(data.index, data.realizedVolatility7, label='RV 7 DTE')

plt.ylabel("Volatility (%)")
plt.legend()
plt.tight_layout()

In [ ]:
data = amberdata_client.get_volatility_term_structures_richness(exchange='deribit', currency='btc', startDate='2024-01-01', endDate='2024-05-01')
data = pd.DataFrame(data['payload']['data'])
data.index = pd.to_datetime(data.timestamp)

plt.figure(figsize=(15, 7))
plt.plot(data.richness)
plt.title("BTC Term Structure Richness: 2024-01-01 to 2024-05-01")
plt.ylabel("Term Structure Richness Ratio")

In [ ]:
# plot BTC DVOL 
dvol = amberdata_client.get_volatility_index(exchange='deribit', currency='btc', startDate='2024-01-01', endDate='2024-06-01', timeInterval='day')
dvol = pd.DataFrame(dvol['payload']['data'])
dvol.index = pd.to_datetime(dvol.exchangeTimestamp, unit='ms')
dvol = dvol[dvol.currency=='BTC']
dvol = dvol.sort_index()

mpf.plot(dvol, type='candle', style='charles', figratio=(15, 7), 
         ylabel='DVOL Index', axtitle='BTC DVOL: 2024-01-01 to 2024-06-01', tight_layout=True)

In [ ]:
# plot DVOL vs spot and RR vs spot
data = amberdata_client.get_volatility_index_decorated(exchange='deribit', currency='btc',  startDate='2024-01-01', endDate='2024-06-01', timeInterval='day')
data = pd.DataFrame(data['payload']['data'])
data.index = pd.to_datetime(data.exchangeTimestamp, unit='ms')
data = data[data.currency=='BTC']
data = data.sort_index()

plt.figure(figsize=(15, 7))
scatter = plt.scatter(data.indexPrice, data.atm, c=data.daysAgo, cmap='coolwarm')
cbar = plt.colorbar(scatter)
cbar.set_label('Days Ago')
plt.title("BTC ATM IV & Spot: 2024-01-01 to 2024-06-01")
plt.xlabel("Spot Price")
plt.ylabel("ATM IV")

In [ ]:
plt.figure(figsize=(15, 7))
scatter = plt.scatter(data.indexPrice, data.delta25RrSkew, c=data.daysAgo, cmap='coolwarm')
cbar = plt.colorbar(scatter)
cbar.set_label('Days Ago')
plt.title("BTC 25 Delta RR Skew & Spot: 2024-01-01 to 2024-06-01")
plt.xlabel("Spot Price")
plt.ylabel("25 RR Skew")

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(data.close/data.atm)
plt.title("BTC DVOL / ATM 30 DTE IV: 2024-01-01 to 2024-06-01")
plt.ylabel("Butterfly Ratio")

In [ ]:
# get vol metrics table
data = amberdata_client.get_volatility_metrics(exchange='deribit', currency='btc', daysBack=7)
data = pd.DataFrame(data['payload']['data'])
data

In [ ]:
# plot put/call trade distribution
data = amberdata_client.get_trades_flow_block_volumes(exchange='deribit', currency='btc')
data = pd.DataFrame(data['payload']['data'])
data = data.groupby("putCall").sum()

plt.figure(figsize=(15, 7))
labels = ['Calls', 'Puts']
plt.pie(data.contractVolume.values.flatten(), labels=labels, autopct='%1.1f%%')
plt.title("BTC Put Call Trade Distribution Over the Past 7 Days")

In [ ]:
# plot block trade bar plot
data = amberdata_client.get_trades_flow_block_volumes(exchange='deribit', currency='btc', startDate='2024-06-14', endDate='2024-06-21')
data = pd.DataFrame(data['payload']['data'])

data['expirationTimestamp'] = pd.to_datetime(data.expirationTimestamp, unit='ms')
data['adj_volume'] = np.where(data.putCall=='P', -data.contractVolume, data.contractVolume)
# only look at expiries that are valid today 
data = data[data.expirationTimestamp >= pd.to_datetime(time.time(), unit='s')]

exps = set(data.expirationTimestamp)
store = []
for exp in exps:
    sub = data[data.expirationTimestamp == exp].reset_index(drop=True)    
    sub = sub[['expirationTimestamp', 'strike', 'putCall', 'adj_volume']]
    # filter out strikes to allow for better visual plotting
    sub = sub[(sub.strike > 60_000) & (sub.strike < 80_000)]
    sub['maturity'] = exp
    store.append(sub)
    
combined_df = pd.concat(store)
pivot_df = combined_df.pivot_table(index='strike', columns='maturity', values='adj_volume', aggfunc='sum').fillna(0)    

fig, ax = plt.subplots(figsize=(20, 10))
# Define colors for each maturity
colors = sns.color_palette("husl", len(pivot_df.columns))
maturities = pivot_df.columns

# Stack the bars
bottom = pd.Series([0]*len(pivot_df), index=pivot_df.index)
for maturity, color in zip(maturities, colors):
    sns.barplot(x=pivot_df.index, y=pivot_df[maturity], bottom=bottom, label=maturity, color=color)
    bottom += pivot_df[maturity]

ax.legend(title='Maturities')    
ax.set_xlabel('Strike Price')
ax.set_ylabel('Contract Volume')
yticks = ax.get_yticks()
yticklabels = ['C' + str(y) if y > 0 else 'P' + str(-y) if y < 0 else '0' for y in yticks]
ax.set_yticklabels(yticklabels);
plt.title("BTC Block Volume Traded: 2024-06-14 to 2024-06-21");

In [ ]:
# plot top 10 instruments traded
data = amberdata_client.get_instruments_most_traded(exchange='deribit', currency='BTC', startDate='2024-06-07', endDate='2024-06-12')
data = pd.DataFrame(data['payload']['data'])

plt.figure(figsize=(15, 7))
plt.barh(data.instrument[:10], data.contractVolume[:10][::-1])
plt.title("BTC Top 10 Instruments Traded: 2024-06-07 to 2024-06-12")
plt.xlabel("Number of Contracts Traded");

In [ ]:
# plot trade distribution
data = amberdata_client.get_trades_flow_put_call_distribution(exchange='deribit', currency='BTC', startDate='2024-06-07', endDate='2024-06-12')
data = pd.DataFrame(data['payload']['data'])

plt.figure(figsize=(15, 7))
labels = ['Calls Bought', 'Puts Bought', 'Calls Sold', 'Puts Sold']
plt.pie(data.iloc[:, :4].values.flatten(), labels=labels, autopct='%1.1f%%')
plt.title("BTC Put Call Trade Distribution: 2024-06-07 to 2024-06-12");

In [ ]:
# plot normalized gamma USD
data = amberdata_client.get_trades_flow_gamma_exposures_normalized_usd(exchange='deribit', currency='btc')
data = pd.DataFrame(data['payload']['data'])
data.index = pd.to_datetime(data.snapshotTimestamp, unit='ms')

plt.figure(figsize=(15, 7))
plt.plot(data.normalizedGammaUSD/1_000_000)
plt.axhline(0, linestyle='--', c='r')
plt.title("BTC Normalized Gamma Exposure USD: 2022-11-17 to 2024-06-13")
plt.ylabel("USD Gamma Exposure ($ Millions)");

In [ ]:
# table of gex snapshots across different instruments
data = amberdata_client.get_trades_flow_gamma_exposures_snapshots(exchange='deribit', currency='btc', startDate='2024-01-01', endDate='2024-01-02')
data = pd.DataFrame(data['payload']['data'])
data

In [ ]:
# plot top 10 largest vega trades
data = amberdata_client.get_options_scanner_top_trades(exchange='deribit', currency='btc', startDate='2024-06-07', endDate='2024-06-12')
data = pd.DataFrame(data['payload']['data'])
# only look at top 10 
data = data.sort_values("sizeVega", ascending=False).head(10).reset_index(drop=True)
# sort by descending order 
plt.figure(figsize=(15, 7))
plt.barh(data.instrument, data.sizeVega[::-1])
plt.title("BTC Largest 10 Vega Option Trades: 2024-06-07 to 2024-06-12")
plt.xlabel("Size Vega")